# 02 RAG | 01 Chat Completion Text | 05 Extensions SDK

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create ChatClient

The `IChatClient` from the Microsoft.Extensions.AI NuGet package offers an abstraction which allows using models from multiple providers.

In [9]:
#r "nuget: Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.3.0-preview.1.25161.3"
#r "nuget: DotNetEnv, 3.1.1"

using Azure;
using Azure.AI.Inference;
using Microsoft.Extensions.AI;
using DotNetEnv;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";    
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";
oAiEndpoint = $"{oAiEndpoint}openai/deployments/{chatCompletionDeploymentName}";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
        
IChatClient chatClient =
    new ChatCompletionsClient(new Uri(oAiEndpoint), azureKeyCredential)
        .AsChatClient(chatCompletionDeploymentName);

Console.WriteLine($"IChatClient created...");


Installed Packages DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.Abstractions, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.AzureAIInference, 9.3.0-preview.1.25161.3

IChatClient created...


## Step 1: Basic Interaction

The following cell, demonstrate a basic interaction using the `chatClient` object. In this case, the system message is used to provide instructions or settings for the assistant, such as its personality or behavior.

Prompt: `
    "You are an AI assistance who helps finding information around international sport events"
    "Who won the Super Bowl 2024?"

Because the GPT-4o model is trained with data up to October 2023 the model will answer with some information that the Super Bowl 2024 hasn't yet taken place.

In [24]:
string prompt = @"
    You are an AI assistance who helps finding information around international sport events
    Who won the Super Bowl 2024?"
;
        
ChatResponse chatResponse = await chatClient.GetResponseAsync(prompt);

Console.WriteLine($"Model response: \n {chatResponse.Text}"); 

Model response: 
 I’m sorry, but I don’t have information on the Super Bowl 2024 since my training data includes information only up until October 2023. You might want to check the latest news sources or sports websites for the most current information on the Super Bowl.


## Step 2: Complex interaction - Chat History

### Prepare chat history

The system prompt is grounded with information regarding the winner and the score of the super bowl 2024

A simulated chat history is provided using a `List<ChatMessage>` with the following information. 

```html
    - Who won the Super Bowl in 2024?
    - Provide just the winning team and the score
    - Don't return other information
    
```



In [20]:
string systemMessage = @"
    You are an AI assistance who helps finding information around international sport events.
    To answer questions you just take information provided to you!
    You don't use your training data.
    The Kansas City Chiefs won the Super Bowl 2024.
    The match ended with a score of 38-35.
";

string[] userMessages = new string[] { 
    "Who won the Super Bowl 2024?",
    "Provide just the winning team and the score.",
    "Don't return other information." 
};

List<ChatMessage> chatMessages = new List<ChatMessage>
{
    new ChatMessage(Microsoft.Extensions.AI.ChatRole.System, systemMessage),
    new ChatMessage(Microsoft.Extensions.AI.ChatRole.User, userMessages[0]),
    new ChatMessage(Microsoft.Extensions.AI.ChatRole.User, userMessages[1]),
};

Console.WriteLine($"Chat messages created...");

Chat messages created...


### Model call

The chat history is provided to the model for completion

In [21]:
  
ChatResponse chatResponse = await chatClient.GetResponseAsync(chatMessages);

Console.WriteLine($"Model response: \n {chatResponse.Text}"); 


Model response: 
 Kansas City Chiefs, 38-35.


## Step 3: Streaming response

`GetStreamingResponseAsync()` provides results as stream as soon as they are provided by the model.

In [23]:
Console.WriteLine($"Model response streaming:");
await foreach (ChatResponseUpdate chatResponseUpdate in chatClient.GetStreamingResponseAsync(chatMessages))
{
    Console.Write(chatResponseUpdate.Text);
}

Model response streaming:
Kansas City Chiefs, 38-35.